In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

class disp(object):
    template = '<div style="float: left;padding:10px;"> <b>[{0}]</b> {1}</div>'
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

import pandas as pd
import numpy as np

### [예제1] query() 메서드 이해

* query는 비트연산(&,|,~)과 논리연산(and,or,not) 동일시
* df[df['age'] >= 20 and df['age'] <= 30] -> ERROR
* **df.query('age >= 20 and age <= 30') -> OK!**

In [ ]:
# [1]
# 조건식은 논리연산자와 범위연산 불가능

df = pd.DataFrame({'name':['kim','lee','park','song'], 'age':[20, 40, 35, 25]})

r1 = df[df['age'] >= 20 and df['age'] <= 30] #error. 논리연산 불가
r2 = df[20 <= df['age'] <= 30] #error. 범위연산 불가
r3 = df[(df['age'] >= 20) & (df['age'] <= 30)]

disp('df', 'r3')

In [ ]:
# [2]
# 그러나, query는 논리연산과 범위연산 사용 가능

df = pd.DataFrame({'name':['kim','lee','park','song'], 'age':[20, 40, 35, 25]})

r4 = df.query('age >= 20 & age <= 30') 
r5 = df.query('age >= 20 and age <= 30')
r6 = df.query('20 <= age <= 30')

disp('r4', 'r5', 'r6')

In [ ]:
# [3]
# 조건식은 in, not in 연산자 불가능

df = pd.DataFrame({'name':['kim','lee','park','song'], 'age':[20, 40, 35, 25]})

r7 = df[df['name'] in ['kim', 'park']] #error
r8 = df[(df['name'] == 'kim') | (df['name'] == 'park')]
r9 = df[df['name'].isin(['kim', 'park'])]

disp('r8', 'r9')

In [ ]:
# [4]
# query는 in, not in 연산 가능

df = pd.DataFrame({'name':['kim','lee','park','song'], 'age':[20, 40, 35, 25]})

r10 = df.query('name in ["kim", "park"]')
r11 = df.query('name == ["kim", "park"]')
r12 = df.query('name.isin(["kim", "park"])', engine='python')

disp('r10', 'r11', 'r12')

### [예제2] query() 메서드 이해2

In [ ]:
# [1]

d = {'class':['A','B','A','C'],'name':['kim','lee','park','song'],'id':range(11,15),'exam':[80,50,90,60]}
df = pd.DataFrame(d).set_index('id')

r1 = df[df.index == 11]
r2 = df.query('id == 11')
r3 = df.query('index == 11') 

disp('df', 'r1', 'r2', 'r3')

In [ ]:
# [2]

d = {'class':['A','B','A','C'],'name':['kim','lee','park','song'],'id':range(11,15),'exam':[80,50,90,60]}
df = pd.DataFrame(d).set_index('id')

x, y = df['class'], 'A'
r4 = df.query('class == "A"') #error
r5 = df.query('`class` == "A"')
r6 = df.query('@x == @y')

disp('df', 'r5', 'r6')

In [ ]:
# [3]

d = {'class':['A','B','A','C'],'name':['kim','lee','park','song'],'id':range(11,15),'exam':[80,50,90,60]}
df = pd.DataFrame(d).set_index('id')

x = df['exam'].mean()
y = ['A', 'C']
r7 = df.query('exam <= @x')
r8 = df.query('`class` in @y ')['exam'].max()

disp('df','r7'); r8

### [예제3] filter() 메서드 이해

In [ ]:
# [1]

d = {'kor':[80,50,90,60],'eng':[70,100,80,50]}
df = pd.DataFrame(d, index=['kim','lee','park','ki'])

r1 = df.filter(items = ['kor'])
r2 = df.filter(like = 'k')

disp('df', 'r1', 'r2')

In [ ]:
# [2]

d = {'kor':[80,50,90,60],'eng':[70,100,80,50]}
df = pd.DataFrame(d, index=['kim','lee','park','ki'])

r3 = df.filter(items = ['kim', 'ki'], axis=0)
r4 = df.filter(regex = '^k', axis=0) 
r5 = df.loc[df.index.isin(['kim', 'ki'])]
r6 = df[ df.index.str.startswith('k') ]

disp('df', 'r3', 'r4', 'r5', 'r6')

In [ ]:
# [3]

d = {'kor':[80,50,90,60],'eng':[70,100,80,50]}
df = pd.DataFrame(d, index=['kim','lee','park','ki'])

r7 = df.filter(regex='a$', like='K', axis=0) #error

In [ ]:
# [4]

d = {'kor':[80,50,90,60],'eng':[70,100,80,50]}
df2 = pd.DataFrame(d, index=['kim','lee','park','kim'])

r8 = df2.filter(items=['kim'], axis=0) #error
r9 = df2.filter(like='m', axis=0)
r10 = df2.filter(regex='^k', axis=0)

disp('df2', 'r9', 'r10')

### [예제4] sr.apply() 메서드 이해 1

**sr.apply()**
* series의 각 values에 지정 함수를 일괄 적용

**df.apply()**
* df의 axis에 따른 series에 대하여 함수를 일괄 적용
* axis = 0 : 행방향 / axis = 1: 열방향

**df.applymap()**
* df의 각 values에 지정 함수를 일괄 적용

In [3]:
# [1]
# pandas의 round는 문자열 형태로 전달

df = pd.DataFrame({'A': [2, 5, 9], 'B':[10.5, 20.5, 15.5]})

r1 = df['B'].round()        # pandas 함수. 10.0, 20.0, 16.0
r2 = df['B'].apply('round') # pandas 함수. 10.0, 20.0, 16.0

r3 = df['B'].apply(round)   # python built-in 함수 사용. 10, 20, 16

r1; r2; r3

0    10.0
1    20.0
2    16.0
Name: B, dtype: float64

0    10.0
1    20.0
2    16.0
Name: B, dtype: float64

0    10
1    20
2    16
Name: B, dtype: int64

In [5]:
# [2]

df = pd.DataFrame({'A': [2, 5, 9], 'B':[10.5, 20.5, 15.5]})

r4 = df['A'].apply(np.square)
r5 = df['A'].apply('mean')
# r6 = df['A'].apply(mean) #error. python built-in 함수에 없음
r4; r5

0     4
1    25
2    81
Name: A, dtype: int64

5.333333333333333

In [2]:
# [3]

df = pd.DataFrame({'A': [2, 5, 9], 'B':[10.5, 20.5, 15.5]})

def func(x):
    print(type(x), x)
    return x*x

r7 = df['A'].apply(func)  # 4, 25, 81 series 반환
r7

<class 'int'> 2
<class 'int'> 5
<class 'int'> 9


0     4
1    25
2    81
Name: A, dtype: int64

### [예제5] sr.apply() 메서드 이해 2

In [4]:
# [1]

df = pd.DataFrame({'model':['TV','PC','HP'], 'price':[400,200,100]})

def func1(t, x, y, z):
    return t * x + y - z

r1 = df['price'].apply(func1, x=0.5, y=50, z=5)    # t = df['price']
r2 = df['price'].apply(func1, args=(0.5, 50, 5))   # arg를 통해 function의 parameter 전달
r3 = df['price'].apply(func1, args=[0.5, 50], z=5)
r4 = df['price'].apply(func1, args=(0.5,), y=50, z=5)
df; r1; r2; r3; r4

,model,price
0,TV,400
1,PC,200
2,HP,100


0    245.0
1    145.0
2     95.0
Name: price, dtype: float64

0    245.0
1    145.0
2     95.0
Name: price, dtype: float64

0    245.0
1    145.0
2     95.0
Name: price, dtype: float64

0    245.0
1    145.0
2     95.0
Name: price, dtype: float64

In [ ]:
# [2]

df = pd.DataFrame({'model':['TV','PC','HP'], 'price':[400,200,100]})
df

def func2(x, y):
    return x-50 if x<y else x-100

df['sale'] = df['price'].apply(func2, y=300)
df

### [예제6] sr.apply() 활용 예

In [ ]:
d = {'model':['TV','COMPUTER','PHONE'],'price':[4000000,2000000,1000000]}
df = pd.DataFrame(d)

r1 = df['model'].apply(lambda x: x if len(x)<=4 else (x[:3]+'~'))
r2 = df['price'].apply('{:,}원'.format)
df; r1; r2

### [예제7] df.apply() 이해 1

In [10]:
df = pd.DataFrame({'A': [1, 2, 3], 'B':[4, 5, 6]})

def f1(x):
    print(x)
    return x*x

r1 = df.apply(f1)  
r1

0    1
1    2
2    3
Name: A, dtype: int64
0    4
1    5
2    6
Name: B, dtype: int64


,A,B
0,1,16
1,4,25
2,9,36


### [예제8] df.apply() 이해 2

In [5]:
# [1]

df = pd.DataFrame({'A': [1, 2, 3], 'B':[4, 5, 6], 'C':[7, 8, 9]})

r1 = df.apply(np.sum)          # axis=0. 행방향으로 sum
r2 = df.apply(np.sum, axis=1)  # axis=1. 열방향으로 sum
df; r1; r2

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


A     6
B    15
C    24
dtype: int64

0    12
1    15
2    18
dtype: int64

In [6]:
# [2]

df = pd.DataFrame({'A': [1, 2, 3], 'B':[4, 5, 6], 'C':[7, 8, 9]})

r3 = df.apply(np.square)
r4 = df.apply(lambda x: x['A']*x['B']+x['C'], axis=1)

disp('df', 'r3'); r4

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9
,A,B,C
0,1,16,49
1,4,25,64
2,9,36,81


0    11
1    18
2    27
dtype: int64

### [예제9] df.apply(): result_type 옵션

axis=1(column)일 때만 적용 -> 즉, 결과가 행 series로 나오는 경우만 적용 가능
* None(default) : 함수의 반환 타입 따름 (단, list-like인 경우 series로 반환)
* reduce : 함수 반환 타입이 list-like인 경우 series로 반환
* expand : 함수 반환 타입이 list-like인 경우 column으로 확장
* broadcast : 원래 dataframe 모양으로 broadcast됨

In [17]:
# [1]

df = pd.DataFrame({'A': [100,50,70], 'B':[90,80,20], 'C':[10,60,40]}, index=[1,2,3]) 

func1 = lambda x:[ x.sum(), x.max(), x.min()]

r1 = df.apply(func1, axis=1)  # 열방향으로 함수 적용 후, series로 반환
r2 = df.apply(func1, axis=1, result_type='reduce')  # r2와 같음

# series로 반환된 후, column 확장 -> 그래서 컬럼 이름이 0,1,2
r3 = df.apply(func1, axis=1, result_type='expand')

# 원래 dataframe 모양으로 broadcast
r4 = df.apply(func1, axis=1, result_type='broadcast')

r1; r2
disp('df', 'r3', 'r4')

1    [200, 100, 10]
2     [190, 80, 50]
3     [130, 70, 20]
dtype: object

1    [200, 100, 10]
2     [190, 80, 50]
3     [130, 70, 20]
dtype: object

,A,B,C
1,100,90,10
2,50,80,60
3,70,20,40
,0,1,2
1,200,100,10
2,190,80,50
3,130,70,20
,A,B,C
1,200,100,10
2,190,80,50


In [19]:
# [2]

df = pd.DataFrame({'A': [100,50,70], 'B':[90,80,20], 'C':[10,60,40]}, index=[1,2,3]) 

func2 = lambda x:[ x.max(), x.min()]
r5 = df.apply(func2, axis=1, result_type='expand')
# r6 = df.apply(func2, axis=1, result_type='broadcast') # error

disp('df', 'r5')

### [예제10] applymap()의 이해

In [9]:
# df의 각 value에 함수 적용

df = pd.DataFrame({'A': [0.07, 0.234, 0.68], 'B':[0.78,0.95,0.05], 'C':[0.88,0.95,0.35]}, index=[1,2,3]) 
df

r1 = df.applymap(round)
r2 = df.applymap('{:.1%}'.format )
# r3 = df.apply('{:.1%}'.format ) # error
r1; r2

,A,B,C
1,0.070,0.78,0.88
2,0.234,0.95,0.95
3,0.680,0.05,0.35


,A,B,C
1,0,1,1
2,0,1,1
3,1,0,0


,A,B,C
1,7.0%,78.0%,88.0%
2,23.4%,95.0%,95.0%
3,68.0%,5.0%,35.0%


### [예제11] map() 메서드의 이해

* pandas와 python의 map은 다름
* pandas map : series의 각 value들에 대하여 특정 동작을 수행
* python map : 배열 내의 모든 요소 각각에 대하여 주어진 함수를 호출한 결과를 모아 새로운 배열을 반환

In [5]:
d = {'model':['tv','com','audio','phone'], 'price':[200,300,150,450]}
df = pd.DataFrame(d)

code = {'tv':'TV','com':'PC','phone':'HP','audio':'AD'}
r1 = df['price'].map(lambda x:'high' if x>=300 else'low')
r2 = df['price'].map(float)
r3 = df['model'].map(pd.Series(code))
r4 = df['model'].map(code)
df; r1; r2; r3; r4

,model,price
0,tv,200
1,com,300
2,audio,150
3,phone,450


0     low
1    high
2     low
3    high
Name: price, dtype: object

0    200.0
1    300.0
2    150.0
3    450.0
Name: price, dtype: float64

0    TV
1    PC
2    AD
3    HP
Name: model, dtype: object

0    TV
1    PC
2    AD
3    HP
Name: model, dtype: object

### [예제12] map()을 이용한 mapping 활용

In [ ]:
# [1]

d = {'model':['tv','com','audio','phone'], 'price':[np.nan,300,150,450]}
df = pd.DataFrame(d)

code = {'tv':'TV','com':'PC','audio':'AD'}
r1 = df['model'].map(code)
r2 = df['model'].replace(code)
r3 = df['model'].apply(code.get)
r4 = df['model'].apply(code.get, args = (0,))
df; r1; r2; r3; r4

In [ ]:
# [2]

d = {'model':['tv','com','audio','phone'], 'price':[np.nan,300,150,450]}
df = pd.DataFrame(d)

r5 = df['price'].map('{:,}만원'.format)
r6 = df['price'].map('{:,}만원'.format, na_action='ignore')
r7 = df['price'].fillna(0).map('{:,}만원'.format)
df; r5; r6; r7

### [예제13] where(), mask() 메서드의 이해

* where : 조건이 거짓인 행 또는 값 변경
* mask  : 조건이 참인 행 또는 값 변경

In [6]:
# [1]

df = pd.DataFrame({'model':['TV', 'PC', 'HP', 'AD'], 'price':[200,300,150,450]})

r1 = df['price'].where(df['price'] >= 300, 'low') # price 300 이상이 아니면 low로 변경
r2 = df['price'].mask(df['price'] >= 300, 'high') # price 300 이상이면 high로 변경
r3 = df['price'].where(df['price'] >= 300)
r4 = np.where(df['price']>=300, 'high', 'low')
df; r1; r2; r3; r4

,model,price
0,TV,200
1,PC,300
2,HP,150
3,AD,450


0    low
1    300
2    low
3    450
Name: price, dtype: object

0     200
1    high
2     150
3    high
Name: price, dtype: object

0      NaN
1    300.0
2      NaN
3    450.0
Name: price, dtype: float64

array(['low', 'high', 'low', 'high'], dtype='<U4')

In [7]:
# [2]

df = pd.DataFrame({'model':['TV', 'PC', 'HP', 'AD'], 'price':[200,300,150,450]})
    
r5 = df.mask(df['price']<=200, '*')

f1 = lambda x : x['price'] <= 200
sr = pd.Series(['-', 0], index=['model','price'])
r6 = df.mask(f1, sr, axis=1)
df; r5; r6

,model,price
0,TV,200
1,PC,300
2,HP,150
3,AD,450


,model,price
0,*,*
1,PC,300
2,*,*
3,AD,450


,model,price
0,-,0
1,PC,300
2,-,0
3,AD,450
